In [141]:
from IPython.display import HTML

"""Cash ladder

Demonstration of how to compute a cash ladder for a portfolio.

Attributes
----------
cash
"""

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [142]:
import warnings
warnings.filterwarnings('ignore')

In [143]:
# Import general purpose packages
import os
import json
from datetime import datetime, timedelta
import pytz
#from get_cashforecast import cash_forecast

# Import lusid specific packages
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id

# Import data wrangling packages
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
# secrets_path = os.getenv("FBN_SECRETS_PATH")
secrets_path = os.getenv("HOME") + "/secrets/fbn-local.json"
print(secrets_path)

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

/Users/peternagymathe/secrets/fbn-local.json


In [144]:
## DATA
with open("config/cash_ladder_seed_data.json", "r") as seed_data_config:
    mapping_file = json.loads(seed_data_config.read())
    
# Create a new scope

scope = "cash-ladder-001"
portfolio_code = "EQUITY_FUND"

# Load a file of equity transactions

transactions_file = r"data/cash_ladder.csv"
transactions_df = pd.read_csv(transactions_file)
transactions_df["portfolio_code"] = portfolio_code

# Load portfolios, instruments, and transactions

seed_data_response = seed_data(
    api_factory,
    ["portfolios", "instruments", "transactions"],
    scope,
    transactions_file,
    "csv",
    mappings = mapping_file
)

In [145]:
from lusid.api import PortfoliosApi, TransactionPortfoliosApi
from lusid.models import CreateTransactionPortfolioRequest, Portfolio
portfolios_api: PortfoliosApi = api_factory.build(PortfoliosApi)
transaction_portfolios_api: TransactionPortfoliosApi = api_factory.build(TransactionPortfoliosApi)
def get_or_create_portfolio(scope, code) -> Portfolio:
    portfolio: Portfolio = None
    try:
        portfolio = portfolios_api.get_portfolio(scope, portfolio_code)
    except:
        portfolio = transaction_portfolios_api.create_portfolio(scope, CreateTransactionPortfolioRequest(
            portfolio_code, 
            "description",
            portfolio_code,
            datetime(1900, 1, 1, tzinfo=pytz.UTC).isoformat(),
            'USD',
            instrument_scopes= [scope]))
    return portfolio

portfolio = get_or_create_portfolio(scope, portfolio_code)



# Upserting Recipe

We need to upsert a recipe to be able to use upserted quotes -> dependency

In [146]:
from lusid.api import ConfigurationRecipeApi
from lusid.models import configuration_recipe, market_context, market_data_key_rule, UpsertRecipeRequest, ResourceId
configuration_api: ConfigurationRecipeApi = api_factory.build(ConfigurationRecipeApi)
recipe_id =  ResourceId(scope, "recipe-code-001")
recipe_response = configuration_api.upsert_configuration_recipe(UpsertRecipeRequest(
  configuration_recipe = configuration_recipe.ConfigurationRecipe(
    recipe_id.scope, 
    recipe_id.code, 
    market = market_context.MarketContext([
      market_data_key_rule.MarketDataKeyRule("Quote.*.*", "Lusid", scope, "Price", "mid", "30D"),
      market_data_key_rule.MarketDataKeyRule("Quote.ClientInternal.*", "Lusid", scope, "Price", "mid", "30D"),
    ]))))
print(recipe_response)


{'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://localhost.lusid.com:8282/app/insights/logs/0HMOALRS82CUQ:0000000E',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'value': datetime.datetime(2023, 2, 9, 15, 7, 36, 420367, tzinfo=tzutc())}


# Upsert Quotes

In [147]:
from lusid.api import TransactionPortfoliosApi, QuotesApi
from lusid.models import UpsertQuoteRequest, QuoteId, QuoteSeriesId, MetricValue, BucketedCashFlowRequest, ResourceId
transaction_portfolio_api: TransactionPortfoliosApi = api_factory.build(TransactionPortfoliosApi)
quotes_api: QuotesApi = api_factory.build(QuotesApi)

def create_quote_request(instrument_id: str, instrument_id_type: str, value: float, currency: str, field: str = "mid", effective_at: datetime = datetime.now(pytz.utc)):
    return UpsertQuoteRequest(
            quote_id=QuoteId(
                quote_series_id=QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=instrument_id, # row["client_internal"],
                    instrument_id_type=instrument_id_type, # "ClientInternal",
                    quote_type="Price",
                    field=field,
                ),
                effective_at=effective_at.isoformat(),
            ),
            metric_value=MetricValue(value=value, unit=currency),
            scale_factor=100,
        )

instruments = transactions_df.loc[transactions_df["instrument_type"] != "cash", ["instrument_id", "currency", "instrument_type"]].drop_duplicates(["instrument_id"]) # get non cash instruments
print(instruments)

quote_reqs: pd.DataFrame = instruments.apply(lambda row: create_quote_request(row["instrument_id"], "ClientInternal", 1000, row["currency"]), axis=1)
print("upsert quotes to scope: ", scope)
quote_upsert_response = quotes_api.upsert_quotes(scope, request_body = quote_reqs.to_dict())
print(quote_upsert_response.failed)

   instrument_id currency instrument_type
0       EQ_GB001      GBP          equity
3       EQ_GB002      GBP          equity
5       EQ_GB003      GBP          equity
6       EQ_GB004      GBP          equity
13      EQ_US001      USD          equity
14      EQ_US002      USD          equity
15      EQ_US003      USD          equity
16      EQ_US004      USD          equity
upsert quotes to scope:  cash-ladder-001
{}


# Query raw cashflows
Note: This will return an empty array due to not having any instrument cashflows only fund movements

In [148]:
from lusid.api import TransactionPortfoliosApi
from lusid.models import QueryCashFlowsRequest, PortfolioEntityId
transaction_portfolio_api: TransactionPortfoliosApi = api_factory.build(TransactionPortfoliosApi)
dates = pd.to_datetime(transactions_df["txn_trade_date"]).map(lambda x: x.replace(day = 1).tz_localize('UTC')).map(lambda x: pd.Timestamp.isoformat(x))

cashflow_response = transaction_portfolio_api.get_portfolio_cash_flows(
    scope, 
    portfolio_code,
    recipe_id_scope= recipe_id.scope,
    recipe_id_code= recipe_id.code,
)
print(cashflow_response)

{'href': 'https://localhost.lusid.com:8282/api/transactionportfolios/cash-ladder-001/EQUITY_FUND/cashflows/01%2F01%2F0001%2000%3A00%3A00%20%2B00%3A0031%2F12%2F9999%2023%3A59%3A59%20%2B00%3A00?recipeIdCode=recipe-code-001&recipeIdScope=cash-ladder-001',
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://localhost.lusid.com:8282/app/insights/logs/0HMOALRS82CUQ:00000010',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'next_page': None,
 'previous_page': None,
 'values': []}


# CashLadder display

In [149]:
# from cashladder import get_cash_ladder
# from datetime import timezone
# start_date = datetime(2020, 4, 1, tzinfo=pytz.UTC)
# cladder = get_cash_ladder(start_date, portfolio_code, "GBP", scope)

from lusid.models import ResourceListOfPortfolioCashLadder
dates = pd.to_datetime(transactions_df["txn_trade_date"]).map(lambda x: x.tz_localize('UTC'))
result: ResourceListOfPortfolioCashLadder = transaction_portfolio_api.get_portfolio_cash_ladder(
    scope, 
    portfolio_code, 
    dates.min().isoformat(), 
    datetime.now(pytz.UTC).isoformat(), 
    datetime.now(pytz.UTC).isoformat(),
    recipe_id_code= recipe_id.code,
    recipe_id_scope= recipe_id.scope)
old_cash_ladder = result.values
print(old_cash_ladder)

[{'currency': 'USD',
 'failed': {},
 'links': None,
 'records': [{'activities': {'Side1': 20000000.0},
              'close': 20000000.0,
              'effective_date': datetime.datetime(2020, 3, 30, 0, 0, tzinfo=tzutc()),
              'open': 0.0},
             {'activities': {'Side2': -3000000.0},
              'close': 17000000.0,
              'effective_date': datetime.datetime(2020, 4, 1, 0, 0, tzinfo=tzutc()),
              'open': 20000000.0},
             {'activities': {'Side1': -2000000.0, 'Side2': 0.0},
              'close': 15000000.0,
              'effective_date': datetime.datetime(2020, 4, 3, 0, 0, tzinfo=tzutc()),
              'open': 17000000.0},
             {'activities': {'Side2': -3000000.0},
              'close': 12000000.0,
              'effective_date': datetime.datetime(2020, 4, 4, 0, 0, tzinfo=tzutc()),
              'open': 15000000.0}],
 'sub_holding_keys': {}}, {'currency': 'GBP',
 'failed': {},
 'links': None,
 'records': [{'activities': {'Side1': 

# Cashflow bucketing
- Note: We only have fund movements and no instrument cash flows
- Note: This will return an empty due to default bucketing only being applied on instrument cashflows.

In [156]:
from lusid.api import TransactionPortfoliosApi, QuotesApi
from lusid.models import UpsertQuoteRequest, QuoteId, QuoteSeriesId, MetricValue, BucketedCashFlowRequest, ResourceId, BucketedCashFlowResponse
from typing import List
transaction_portfolio_api: TransactionPortfoliosApi = api_factory.build(TransactionPortfoliosApi)
quotes_api: QuotesApi = api_factory.build(QuotesApi)
    

dates = pd.to_datetime(transactions_df["txn_trade_date"]).map(lambda x: x.tz_localize('UTC'))
start_date = dates.min()
# bucketing_dates: np.ndarray = np.array([start_date.replace(month=(start_date.month + i) % 12 + 1, year= start_date.year + i // 12).isoformat() for i in range(12)]) # monthly
bucketing_dates: np.ndarray = np.array([(start_date + timedelta(days = i)).isoformat() for i in range(365)])
# print(bucketing_dates.map(lambda x: pd.Timestamp.isoformat(x)).unique())

def get_bucketed_cashflows(scope: str, code: str, 
    bucketing_dates: List[datetime], 
    roundingKey: str = "RoundDown", 
    groupby: List[str] = None, 
    addresses: List[str] = None,
    cash_flow_type: str = "PortfolioCashFlow") -> BucketedCashFlowResponse:

    return transaction_portfolio_api.get_bucketed_cash_flows(
        scope = scope,
        code = code,
        bucketed_cash_flow_request= BucketedCashFlowRequest(
            roundingKey,
            np.unique(bucketing_dates).tolist(),
            None,
            effective_at=datetime.now(pytz.UTC).isoformat(),
            group_by= groupby,
            addresses= addresses,
            recipe_id= recipe_id,
            cash_flow_type=cash_flow_type
        )
    )

def get_cash_ladder_from_bucketing(scope: str, code: str, bucketing_dates: List[datetime]):
    roundingKey = "RoundDown"
    groupby = ["Valuation/CashFlowDate/RoundDown", "Valuation/SubHoldingKey", "Valuation/CashFlowCurrency"]
    addresses = ["Valuation/CashFlowAmount", "Valuation/OpenBalance", "Valuation/CloseBalance", "Valuation/ActivityType", "Valuation/Movements"]
    return transaction_portfolio_api.get_bucketed_cash_flows(
        scope = scope,
        code = code,
        bucketed_cash_flow_request= BucketedCashFlowRequest(
            roundingKey,
            np.unique(bucketing_dates).tolist(),
            None,
            effective_at=datetime.now(pytz.UTC).isoformat(),
            group_by= groupby,
            addresses= addresses,
            recipe_id= recipe_id,
            window_start=datetime(1, 1, 2, tzinfo=pytz.UTC).isoformat(),
            cash_flow_type="PortfolioCashFlow"
        )
    )

# bucketing_response = get_bucketed_cashflows(scope, portfolio_code, bucketing_dates)
print(bucketing_dates)
bucketing_response = get_cash_ladder_from_bucketing(scope, portfolio_code, bucketing_dates)
print(bucketing_response.failed)

['2020-01-04T00:00:00+00:00' '2020-01-05T00:00:00+00:00'
 '2020-01-06T00:00:00+00:00' '2020-01-07T00:00:00+00:00'
 '2020-01-08T00:00:00+00:00' '2020-01-09T00:00:00+00:00'
 '2020-01-10T00:00:00+00:00' '2020-01-11T00:00:00+00:00'
 '2020-01-12T00:00:00+00:00' '2020-01-13T00:00:00+00:00'
 '2020-01-14T00:00:00+00:00' '2020-01-15T00:00:00+00:00'
 '2020-01-16T00:00:00+00:00' '2020-01-17T00:00:00+00:00'
 '2020-01-18T00:00:00+00:00' '2020-01-19T00:00:00+00:00'
 '2020-01-20T00:00:00+00:00' '2020-01-21T00:00:00+00:00'
 '2020-01-22T00:00:00+00:00' '2020-01-23T00:00:00+00:00'
 '2020-01-24T00:00:00+00:00' '2020-01-25T00:00:00+00:00'
 '2020-01-26T00:00:00+00:00' '2020-01-27T00:00:00+00:00'
 '2020-01-28T00:00:00+00:00' '2020-01-29T00:00:00+00:00'
 '2020-01-30T00:00:00+00:00' '2020-01-31T00:00:00+00:00'
 '2020-02-01T00:00:00+00:00' '2020-02-02T00:00:00+00:00'
 '2020-02-03T00:00:00+00:00' '2020-02-04T00:00:00+00:00'
 '2020-02-05T00:00:00+00:00' '2020-02-06T00:00:00+00:00'
 '2020-02-07T00:00:00+00:00' '2

# Reformat to CashLadder

In [157]:
def format_result0D_column(values: pd.Series):
    return [(x['value'], x["units"]) for x in values]

data = pd.DataFrame.from_records(np.array(bucketing_response.data))
# print(data["Append(Valuation/CashFlowAmount)"].iloc[1])
data["Append(Valuation/CashFlowAmount)"] = data["Append(Valuation/CashFlowAmount)"].map(format_result0D_column)
data["Valuation/SubHoldingKey"] = data["Valuation/SubHoldingKey"].map(lambda x: "; ".join([str(y) for y in x]))
data.sort_values("Valuation/CashFlowCurrency", ascending=False, inplace=True)
display(data)

,Valuation/CashFlowDate/RoundDown,Append(Valuation/CashFlowAmount),Sum(Valuation/CashFlowAmount),SumCumulativeInAdvance(Valuation/CashFlowAmount),SumCumulativeInArrears(Valuation/CashFlowAmount),Valuation/ActivityType,Append(Valuation/Movements),Valuation/SubHoldingKey,Valuation/CashFlowCurrency,Valuation/OpenBalance,Valuation/CloseBalance
0,2020-01-04T00:00:00.0000000+00:00,"[(0.0, (USD))]",0.0,0.0,NaN,Open,[Open],,USD,0.0,0.0
1,2020-03-30T00:00:00.0000000+00:00,"[(20000000.0, (USD))]",20000000.0,20000000.0,0.0,Activity,[Side1],,USD,0.0,20000000.0
2,2020-04-01T00:00:00.0000000+00:00,"[(-3000000.0, (USD))]",-3000000.0,17000000.0,20000000.0,Activity,[Side2],,USD,20000000.0,17000000.0
3,2020-04-03T00:00:00.0000000+00:00,"[(0.0, (USD)), (-2000000.0, (USD))]",-2000000.0,15000000.0,17000000.0,Activity,"[Side2, Side1]",,USD,17000000.0,15000000.0
4,2020-04-04T00:00:00.0000000+00:00,"[(-3000000.0, (USD))]",-3000000.0,12000000.0,15000000.0,Activity,[Side2],,USD,15000000.0,12000000.0
5,2021-01-02T00:00:00.0000000+00:00,"[(0.0, (USD))]",0.0,12000000.0,12000000.0,Closed,[Closed],,USD,12000000.0,12000000.0
6,2020-01-04T00:00:00.0000000+00:00,"[(0.0, (GBP))]",0.0,0.0,NaN,Open,[Open],,GBP,0.0,0.0
7,2020-03-31T00:00:00.0000000+00:00,"[(500000.0, (GBP))]",500000.0,500000.0,0.0,Activity,[Side1],,GBP,0.0,500000.0
8,2020-04-01T00:00:00.0000000+00:00,"[(-200000.0, (GBP)), (10000000.0, (GBP))]",9800000.0,10300000.0,500000.0,Activity,"[Side2, Side1]",,GBP,500000.0,10300000.0
9,2020-04-03T00:00:00.0000000+00:00,"[(-6900000.0, (GBP)), (-1000000.0, (GBP))]",-7900000.0,2400000.0,10300000.0,Activity,"[Side2, Side1]",,GBP,10300000.0,2400000.0


# Formatting

To have the bucketed cashflows in the same format as cash ladder we just have to do some grouping

In [185]:
from lusid.models import PortfolioCashLadder, CashLadderRecord
# "Valuation/CashFlowCurrency"
grouped = data.groupby(["Valuation/CashFlowCurrency"], sort=False)

def to_record(value: pd.DataFrame) -> CashLadderRecord:
    effective_date = value["Valuation/CashFlowDate/RoundDown"]
    open_balance = value["Valuation/OpenBalance"]
    movements = list(zip(value["Append(Valuation/Movements)"], value["Append(Valuation/CashFlowAmount)"]))
    close_balance = value["Valuation/CloseBalance"]
    return CashLadderRecord(
        effective_date,
        open_balance,
        movements,
        close_balance)

def post_processsing_records(records: List[CashLadderRecord]):
    # not extensive conversion yet
    records = [record for record in records if not (record.open == record.close and len(record.activities) == 1)]
    print("\n".join([str(record) for record in records]))
    return records

def to_cash_ladder(key, df: pd.DataFrame) -> PortfolioCashLadder:
    records = [to_record(row) for (i, row) in df.iterrows()]
    records = post_processsing_records(records)
    return PortfolioCashLadder(key, " ", records)

new_cash_ladder = [to_cash_ladder(key, df) for (key, df) in grouped]
# print(new_cash_ladder)
    

{'activities': [('Side1', (20000000.0, '(USD)'))],
 'close': 20000000.0,
 'effective_date': '2020-03-30T00:00:00.0000000+00:00',
 'open': 0.0}
{'activities': [('Side2', (-3000000.0, '(USD)'))],
 'close': 17000000.0,
 'effective_date': '2020-04-01T00:00:00.0000000+00:00',
 'open': 20000000.0}
{'activities': [('Side2', (0.0, '(USD)')), ('Side1', (-2000000.0, '(USD)'))],
 'close': 15000000.0,
 'effective_date': '2020-04-03T00:00:00.0000000+00:00',
 'open': 17000000.0}
{'activities': [('Side2', (-3000000.0, '(USD)'))],
 'close': 12000000.0,
 'effective_date': '2020-04-04T00:00:00.0000000+00:00',
 'open': 15000000.0}
{'activities': [('Side1', (500000.0, '(GBP)'))],
 'close': 500000.0,
 'effective_date': '2020-03-31T00:00:00.0000000+00:00',
 'open': 0.0}
{'activities': [('Side2', (-200000.0, '(GBP)')),
                ('Side1', (10000000.0, '(GBP)'))],
 'close': 10300000.0,
 'effective_date': '2020-04-01T00:00:00.0000000+00:00',
 'open': 500000.0}
{'activities': [('Side2', (-6900000.0, '(GBP

In [189]:
for (nladder, oladder) in zip(new_cash_ladder, old_cash_ladder):
    nladder: PortfolioCashLadder = nladder
    oladder: PortfolioCashLadder = oladder

    print("=======")
    assert nladder.currency == oladder.currency
    print("From Bucketing: \n", "\n".join([str(r) for r in nladder.records]))
    print("----")
    print("From CashLadder: \n", "\n".join([str(r) for r in oladder.records]))

From Bucketing: 
 {'activities': [('Side1', (20000000.0, '(USD)'))],
 'close': 20000000.0,
 'effective_date': '2020-03-30T00:00:00.0000000+00:00',
 'open': 0.0}
{'activities': [('Side2', (-3000000.0, '(USD)'))],
 'close': 17000000.0,
 'effective_date': '2020-04-01T00:00:00.0000000+00:00',
 'open': 20000000.0}
{'activities': [('Side2', (0.0, '(USD)')), ('Side1', (-2000000.0, '(USD)'))],
 'close': 15000000.0,
 'effective_date': '2020-04-03T00:00:00.0000000+00:00',
 'open': 17000000.0}
{'activities': [('Side2', (-3000000.0, '(USD)'))],
 'close': 12000000.0,
 'effective_date': '2020-04-04T00:00:00.0000000+00:00',
 'open': 15000000.0}
----
From CashLadder: 
 {'activities': {'Side1': 20000000.0},
 'close': 20000000.0,
 'effective_date': datetime.datetime(2020, 3, 30, 0, 0, tzinfo=tzutc()),
 'open': 0.0}
{'activities': {'Side2': -3000000.0},
 'close': 17000000.0,
 'effective_date': datetime.datetime(2020, 4, 1, 0, 0, tzinfo=tzutc()),
 'open': 20000000.0}
{'activities': {'Side1': -2000000.0, '